## 1. 라이브러리 및 설정


In [ ]:
import os
import sys
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import numpy as np
import re
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from pathlib import Path

# OpenCV 사용 가능 여부 확인
try:
    import cv2
    CV2_AVAILABLE = True
    print("✅ OpenCV 사용 가능")
except ImportError:
    CV2_AVAILABLE = False
    print("⚠️ OpenCV가 설치되지 않았습니다. 기본 전처리만 사용합니다.")

# Tesseract 경로 설정 (Windows의 경우 필요할 수 있음)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

print("라이브러리 로드 완료")


## 2. 이미지 전처리 함수들


In [ ]:
def preprocess_basic(image):
    """기본 전처리 (PIL만 사용)"""
    # RGBA -> RGB 변환
    if image.mode == 'RGBA':
        rgb_image = Image.new('RGB', image.size, (255, 255, 255))
        rgb_image.paste(image, mask=image.split()[3])
        image = rgb_image
    elif image.mode != 'RGB':
        image = image.convert('RGB')
    
    # 그레이스케일 변환
    image = image.convert('L')
    
    # 대비 향상
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.5)
    
    # 선명도 향상
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(2.0)
    
    return image

def preprocess_advanced(image):
    """고급 전처리 (OpenCV 사용)"""
    if not CV2_AVAILABLE:
        return preprocess_basic(image)
    
    # RGBA -> RGB 변환
    if image.mode == 'RGBA':
        rgb_image = Image.new('RGB', image.size, (255, 255, 255))
        rgb_image.paste(image, mask=image.split()[3])
        image = rgb_image
    elif image.mode != 'RGB':
        image = image.convert('RGB')
    
    # PIL Image를 numpy array로 변환
    img_array = np.array(image)
    
    # 그레이스케일 변환
    if len(img_array.shape) == 3:
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    else:
        gray = img_array
    
    # 노이즈 제거
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
    
    # 대비 향상 (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(denoised)
    
    # 이진화 (Otsu's method)
    _, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 다시 PIL Image로 변환
    return Image.fromarray(binary)

def resize_image(image, min_size=300):
    """이미지 크기 조정"""
    width, height = image.size
    if width < min_size or height < min_size:
        scale = max(min_size / width, min_size / height)
        new_width = int(width * scale)
        new_height = int(height * scale)
        return image.resize((new_width, new_height), Image.LANCZOS)
    return image

print("전처리 함수 로드 완료")


## 3. OCR 수행 함수


In [ ]:
def perform_ocr(image, psm_mode=6, lang='kor+eng'):
    """
    OCR 수행
    
    Args:
        image: PIL Image 객체
        psm_mode: 페이지 분할 모드 (3=자동, 6=단일 블록, 11=희소 텍스트, 12=단일 텍스트 블록)
        lang: 언어 설정 ('kor+eng', 'kor', 'eng' 등)
    
    Returns:
        tuple: (추출된 텍스트, 신뢰도 정보)
    """
    try:
        custom_config = f'--oem 3 --psm {psm_mode} -l {lang}'
        
        # 텍스트 추출
        text = pytesseract.image_to_string(image, config=custom_config, lang=lang)
        
        # 신뢰도 정보 가져오기
        try:
            data = pytesseract.image_to_data(image, config=custom_config, lang=lang, output_type=pytesseract.Output.DICT)
            confidences = [int(conf) for conf in data['conf'] if int(conf) > 0]
            avg_confidence = sum(confidences) / len(confidences) if confidences else 0
        except:
            avg_confidence = 0
        
        return text.strip(), avg_confidence
    except Exception as e:
        return f"오류 발생: {str(e)}", 0

def test_multiple_psm_modes(image, lang='kor+eng'):
    """여러 PSM 모드로 테스트하고 결과 비교"""
    psm_modes = {
        3: '자동 페이지 분할',
        6: '단일 블록으로 인식',
        11: '희소 텍스트',
        12: 'OSD 없는 단일 텍스트 블록'
    }
    
    results = []
    for psm, desc in psm_modes.items():
        text, confidence = perform_ocr(image, psm_mode=psm, lang=lang)
        results.append({
            'psm': psm,
            'description': desc,
            'text': text,
            'confidence': confidence,
            'text_length': len(text)
        })
    
    return results

print("OCR 함수 로드 완료")


## 4. 이미지 로드 및 테스트


In [ ]:
# 이미지 파일 경로 설정 (여기에 테스트할 이미지 경로를 입력하세요)
image_path = "test_image.jpg"  # 또는 "uploads/your_image.jpg"

# 이미지 로드
if os.path.exists(image_path):
    original_image = Image.open(image_path)
    print(f"✅ 이미지 로드 성공: {image_path}")
    print(f"   크기: {original_image.size}")
    print(f"   모드: {original_image.mode}")
    
    # 원본 이미지 표시
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.imshow(original_image)
    plt.title("원본 이미지")
    plt.axis('off')
    
    # 기본 전처리 결과
    basic_processed = preprocess_basic(original_image.copy())
    plt.subplot(2, 2, 2)
    plt.imshow(basic_processed, cmap='gray')
    plt.title("기본 전처리")
    plt.axis('off')
    
    # 고급 전처리 결과 (OpenCV 사용 가능한 경우)
    if CV2_AVAILABLE:
        advanced_processed = preprocess_advanced(original_image.copy())
        plt.subplot(2, 2, 3)
        plt.imshow(advanced_processed, cmap='gray')
        plt.title("고급 전처리 (OpenCV)")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
else:
    print(f"❌ 이미지를 찾을 수 없습니다: {image_path}")
    print("   위의 image_path 변수를 수정하여 이미지 경로를 지정하세요.")


## 5. OCR 테스트 - 기본 전처리


In [ ]:
if 'original_image' in locals():
    # 기본 전처리된 이미지로 OCR 수행
    basic_processed = preprocess_basic(original_image.copy())
    
    print("=" * 60)
    print("기본 전처리 + OCR 결과")
    print("=" * 60)
    
    # 여러 PSM 모드 테스트
    results = test_multiple_psm_modes(basic_processed, lang='kor+eng')
    
    for result in results:
        print(f"\n[PSM {result['psm']}] {result['description']}")
        print(f"신뢰도: {result['confidence']:.2f}%")
        print(f"텍스트 길이: {result['text_length']}자")
        print(f"\n추출된 텍스트:")
        print("-" * 40)
        print(result['text'])
        print("-" * 40)
    
    # 최고 신뢰도 결과 찾기
    best_result = max(results, key=lambda x: x['confidence'])
    print(f"\n\n🏆 최고 신뢰도 결과: PSM {best_result['psm']} ({best_result['description']})")
    print(f"신뢰도: {best_result['confidence']:.2f}%")
else:
    print("먼저 이미지를 로드하세요.")


## 6. OCR 테스트 - 고급 전처리 (OpenCV)


In [ ]:
if 'original_image' in locals() and CV2_AVAILABLE:
    # 고급 전처리된 이미지로 OCR 수행
    advanced_processed = preprocess_advanced(original_image.copy())
    
    print("=" * 60)
    print("고급 전처리 (OpenCV) + OCR 결과")
    print("=" * 60)
    
    # 여러 PSM 모드 테스트
    results = test_multiple_psm_modes(advanced_processed, lang='kor+eng')
    
    for result in results:
        print(f"\n[PSM {result['psm']}] {result['description']}")
        print(f"신뢰도: {result['confidence']:.2f}%")
        print(f"텍스트 길이: {result['text_length']}자")
        print(f"\n추출된 텍스트:")
        print("-" * 40)
        print(result['text'])
        print("-" * 40)
    
    # 최고 신뢰도 결과 찾기
    best_result = max(results, key=lambda x: x['confidence'])
    print(f"\n\n🏆 최고 신뢰도 결과: PSM {best_result['psm']} ({best_result['description']})")
    print(f"신뢰도: {best_result['confidence']:.2f}%")
elif not CV2_AVAILABLE:
    print("⚠️ OpenCV가 설치되지 않아 고급 전처리를 사용할 수 없습니다.")
    print("   설치: pip install opencv-python")
else:
    print("먼저 이미지를 로드하세요.")


## 7. 전처리 방법 비교


In [ ]:
if 'original_image' in locals():
    print("=" * 60)
    print("전처리 방법 비교")
    print("=" * 60)
    
    # 기본 전처리
    basic_processed = preprocess_basic(original_image.copy())
    basic_text, basic_conf = perform_ocr(basic_processed, psm_mode=6, lang='kor+eng')
    
    print(f"\n[기본 전처리]")
    print(f"신뢰도: {basic_conf:.2f}%")
    print(f"텍스트 길이: {len(basic_text)}자")
    
    # 고급 전처리
    if CV2_AVAILABLE:
        advanced_processed = preprocess_advanced(original_image.copy())
        advanced_text, advanced_conf = perform_ocr(advanced_processed, psm_mode=6, lang='kor+eng')
        
        print(f"\n[고급 전처리 (OpenCV)]")
        print(f"신뢰도: {advanced_conf:.2f}%")
        print(f"텍스트 길이: {len(advanced_text)}자")
        
        # 비교
        print(f"\n\n📊 비교 결과:")
        if advanced_conf > basic_conf:
            print(f"✅ 고급 전처리가 {advanced_conf - basic_conf:.2f}% 더 높은 신뢰도를 보입니다.")
        elif basic_conf > advanced_conf:
            print(f"✅ 기본 전처리가 {basic_conf - advanced_conf:.2f}% 더 높은 신뢰도를 보입니다.")
        else:
            print("두 방법의 신뢰도가 비슷합니다.")
    else:
        print("\n⚠️ OpenCV가 설치되지 않아 고급 전처리를 비교할 수 없습니다.")
else:
    print("먼저 이미지를 로드하세요.")


## 8. 언어 설정 테스트


In [ ]:
if 'original_image' in locals():
    # 전처리된 이미지 준비 (가장 좋은 결과를 낸 전처리 방법 사용)
    if CV2_AVAILABLE:
        test_image = preprocess_advanced(original_image.copy())
        method_name = "고급 전처리"
    else:
        test_image = preprocess_basic(original_image.copy())
        method_name = "기본 전처리"
    
    print("=" * 60)
    print(f"언어 설정 비교 ({method_name})")
    print("=" * 60)
    
    languages = ['kor+eng', 'kor', 'eng']
    
    for lang in languages:
        try:
            text, confidence = perform_ocr(test_image, psm_mode=6, lang=lang)
            print(f"\n[언어: {lang}]")
            print(f"신뢰도: {confidence:.2f}%")
            print(f"텍스트 길이: {len(text)}자")
            print(f"\n추출된 텍스트 (처음 200자):")
            print(text[:200] + "..." if len(text) > 200 else text)
        except Exception as e:
            print(f"\n[언어: {lang}] 오류: {str(e)}")
else:
    print("먼저 이미지를 로드하세요.")


## 9. 사용 팁

### PSM 모드 설명:
- **PSM 3**: 자동 페이지 분할 (복잡한 레이아웃에 적합)
- **PSM 6**: 단일 블록으로 인식 (일반적인 문서에 적합)
- **PSM 11**: 희소 텍스트 (텍스트가 적은 이미지에 적합)
- **PSM 12**: OSD 없는 단일 텍스트 블록 (단순 레이아웃에 적합)

### 전처리 방법:
- **기본 전처리**: PIL만 사용, 빠르고 간단
- **고급 전처리**: OpenCV 사용, 노이즈 제거 및 대비 향상으로 더 정확할 수 있음

### 권장 사항:
1. 여러 이미지로 테스트하여 최적의 설정 찾기
2. 신뢰도가 낮으면 이미지 품질 확인 (해상도, 명확도)
3. 한글+영어 혼합 문서는 `kor+eng` 사용
4. 계약서처럼 구조화된 문서는 PSM 6 또는 3 시도
